<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#isnull" data-toc-modified-id="isnull-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>isnull</a></span><ul class="toc-item"><li><span><a href="#na" data-toc-modified-id="na-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>na</a></span></li><li><span><a href="#outer" data-toc-modified-id="outer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>outer</a></span></li></ul></li></ul></div>

In [1]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [2]:
spark

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('../donnees/meteo30')\
    .cache()

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('../donnees/meteo/postesSynop.csv')  \
      .cache()

@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')


meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 to_timestamp(col('date').cast('string'),'yyyyMMddHHmmss'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','date','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

meteo.select('annee','mois','jour','temperature','humidite',
             'visibilite','pression').show(3)

+-----+----+----+-----------+--------+----------+--------+
|annee|mois|jour|temperature|humidite|visibilite|pression|
+-----+----+----+-----------+--------+----------+--------+
| 2019|  12|   1|        3.7|    0.79|      20.0|  100.86|
| 2019|  12|   1|        2.8|    0.87|     12.23|  101.38|
| 2019|  12|   1|        8.7|    0.75|      10.0|  101.39|
+-----+----+----+-----------+--------+----------+--------+
only showing top 3 rows



In [4]:
villesT.show(3)

+-----+---------------+---------+---------+--------+
|   id|          ville| latitude|longitude|altitude|
+-----+---------------+---------+---------+--------+
|07005|      Abbeville|   50.136|    1.834|      69|
|07015|          Lille|    50.57|   3.0975|      47|
|07020|Pte De La Hague|49.725166|-1.939833|       6|
+-----+---------------+---------+---------+--------+
only showing top 3 rows



# isnull

In [5]:
meteo.where('id < 8000').count()
description = meteo.where('id < 8000')\
     .select('annee','temperature', 'humidite',
             'visibilite', 'pression','precipitations')\
     .describe()\
     .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in
                    ['summary','annee','temperature', 'humidite',
                   'visibilite', 'pression','precipitations']])\
     .show()

+-------+---------+-----------+---------+----------+---------+--------------+
|summary|    annee|temperature| humidite|visibilite| pression|precipitations|
+-------+---------+-----------+---------+----------+---------+--------------+
|  count|2924329.0|  2917134.0|2914795.0| 2524195.0|2911657.0|     2911157.0|
|   mean|   2008.8|      12.69|     0.76|     22.38|    99.57|          0.26|
| stddev|     7.07|       7.33|     0.18|     15.13|     2.53|          1.32|
|    min|   1997.0|      -21.3|     0.01|       0.0|     43.4|          -0.3|
|    max|   2021.0|       46.9|      1.0|     80.01|   105.09|         443.0|
+-------+---------+-----------+---------+----------+---------+--------------+



In [6]:
meteo.where('id < 8000').where(meteo['temperature'].isNull()).count()

7195

In [7]:
meteo.where('id < 8000').where(meteo['humidite'].isNull()).count()

9534

In [8]:
meteo.where('id < 8000').where(meteo['visibilite'].isNull()).count()

400134

In [9]:
meteo.where('id < 8000').where(meteo['pression'].isNull()).count()

12672

In [10]:
meteo.where('id < 8000').where(meteo['precipitations'].isNull()).count()

13172

In [9]:
meteo.where('id < 8000').toPandas().isna().sum()

Py4JJavaError: An error occurred while calling o183.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:377)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:373)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.foreach(SparkPlan.scala:373)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:395)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:394)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$3737/637480592.apply(Unknown Source)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scal

In [13]:
from databricks import koalas as ks
meteoDataFrame.to_koalas().isna().sum()

numer_sta          0
date               0
pmer          169935
tend          574410
cod_tend      601623
dd             72000
ff             70823
t              12680
td             29995
u              31667
vv           1070343
ww           1123757
w1           2089294
w2           2149107
n            1643865
nbas         1714252
hbas         1773164
cl           2212586
cm           2262435
ch           2281730
pres          501908
niv_bar      4012396
geop         3978657
tend24       2805101
tn12         3223289
tn24         3932098
tx12         3217310
tx24         3938527
tminsol      3169016
sw           4102254
tw           4102254
raf10        3209169
rafper       1730167
per          1726348
etat_sol     3350075
ht_neige     3184301
ssfrai       3539650
perssfrai    3555072
rr1          1692290
rr3           666865
rr6          2313648
rr12         2528896
rr24         2772911
phenspe1     2321924
phenspe2     2580434
phenspe3     3466107
phenspe4     4083062
nnuage1      

In [14]:
meteo.where('id < 8000')\
     .where(meteo['temperature'].isNotNull())\
     .where(meteo['humidite'].isNotNull() )\
     .where(meteo['visibilite'].isNotNull() )\
     .where(meteo['pression'].isNotNull() )\
     .count()

2511411

## na

In [15]:
meteo.where('id < 8000')\
     .na.fill(0 ,["precipitations"])\
     .na.drop()\
     .count()

2511411

In [16]:
meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 col('rr1')*3,
                 col('rr3'),
                 col('rr6')/2,
                 col('rr12')/4,
                 col('rr24')/8)\
                   .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression',
                   'precipitations1','precipitations3','precipitations6',
                   'precipitations12','precipitations24')\
             .cache()

In [17]:
meteo.where('id < 8000')\
       .select('precipitations1','precipitations3','precipitations6',
             'precipitations12','precipitations24','temperature')\
       .toDF('prec1','prec3','prec6','prec12','prec24','temp')\
       .describe()\
       .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in['prec1','prec3','prec6',
                          'prec12','prec24','temp']])\
      .show()

+---------+---------+---------+---------+--------+---------+
|    prec1|    prec3|    prec6|   prec12|  prec24|     temp|
+---------+---------+---------+---------+--------+---------+
|1998601.0|2849023.0|1295972.0|1290021.0|990775.0|2917134.0|
|     0.24|     0.26|     0.24|     0.25|    0.25|    12.69|
|     1.66|     1.32|     1.03|     0.91|    0.67|     7.33|
|     -0.3|     -0.1|    -0.05|    -0.03|   -0.01|    -21.3|
|    279.0|    443.0|    104.0|    225.0|   97.66|     46.9|
+---------+---------+---------+---------+--------+---------+



In [18]:
meteo.select( coalesce('precipitations3','precipitations24',
                       'precipitations12','precipitations6',
                       'precipitations1').alias('precipitations')).show()

+--------------+
|precipitations|
+--------------+
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           2.0|
|           2.0|
|          -0.1|
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           0.0|
|           0.6|
|           3.6|
|           1.2|
|           0.0|
+--------------+
only showing top 20 rows



In [19]:
meteo.where('id < 8000')\
     .select( coalesce('precipitations24','precipitations12',
                       'precipitations6','precipitations3',
                       'precipitations1').alias('precipitationsH')
            ).describe().show()

+-------+--------------------+
|summary|     precipitationsH|
+-------+--------------------+
|  count|             2911157|
|   mean|  0.2543271240609963|
| stddev|  1.0554809843183568|
|    min|-0.30000000000000004|
|    max|               220.0|
+-------+--------------------+



In [20]:
meteo.to_koalas().isna().sum()

id                        0
annee                     0
mois                      0
jour                      0
mois_jour                 0
temperature           12680
humidite              31667
visibilite          1070343
pression             501908
precipitations1     1692290
precipitations3      666865
precipitations6     2313648
precipitations12    2528896
precipitations24    2772911
dtype: int64

In [21]:
meteo.where('id < 8000').count()

2924329